In [1]:
import os
import urllib.request
from typing import Literal

import geopandas as gpd
import pandas as pd

from datav import datav_geoatlas

df = pd.read_stata(os.path.expanduser("~/Documents/baseinfo_all_2020_所有注销的工业企业2003-2022(精简版).dta")).query("not 经度_final.isnull()")
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["经度_final"], df["纬度_final"], crs="EPSG:4326"))


def get_adcode(df, level: Literal["province", "city", "district"]="province"):
    china = gpd.read_file(datav_geoatlas(adcode=100000, full=True)).dropna(subset="level").astype(
        {"adcode": int, "childrenNum": int, "subFeatureIndex": int}
    ).drop(columns=["adchar"])
    result = gpd.sjoin(df, china, how="left", predicate="within")
    try_nearest = result.query("name.isnull() and not 经度_final.isnull()")[df.columns]
    nearest = gpd.sjoin_nearest(try_nearest, china, how="left")
    result.loc[try_nearest.index, "adcode"] = nearest["adcode"]
    result.loc[try_nearest.index, "name"] = nearest["name"]
    if level == "province":
        return result[df.columns.tolist() + ["adcode", "name"]]
    results = []
    for pcode, group in result.groupby("adcode"):
        pcode = int(pcode)
        try:
            province_geojson = datav_geoatlas(adcode=pcode, full=True)
        except urllib.request.HTTPError:
            province_geojson = datav_geoatlas(adcode=pcode, full=False)
        province = gpd.read_file(province_geojson)
        province_name = group["name"].iloc[0]
        result = gpd.sjoin(group[df.columns], province, how="left", predicate="within")
        try_nearest = result.query("name.isnull() and not 经度_final.isnull()")[df.columns]
        nearest = gpd.sjoin_nearest(try_nearest, province, how="left", max_distance=0.1)
        result.loc[try_nearest.index, "adcode"] = nearest["adcode"]
        result.loc[try_nearest.index, "name"] = nearest["name"]
        result["province"] = province_name
        results.append(result)
    result = pd.concat(results)
    if level == "city":
        return result[df.columns.tolist() + ["adcode", "name", "province"]]
    # WIP
    raise NotImplementedError

get_adcode(df).astype({"adcode": int})

Skipping field center: unsupported OGR type: 3
Skipping field centroid: unsupported OGR type: 3
Skipping field acroutes: unsupported OGR type: 1
/Users/tcztzy/GitHub/datav/.venv/lib/python3.9/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,RecordID,经度_final,纬度_final,geometry,adcode,name
0,39609536,112.893791,23.156624,POINT (112.89379 23.15662),440000,广东省
1,24044511,118.686211,41.001991,POINT (118.68621 41.00199),130000,河北省
2,38157212,121.343140,31.416922,POINT (121.34314 31.41692),310000,上海市
3,44227634,108.949188,19.233969,POINT (108.94919 19.23397),460000,海南省
4,10850108,116.125328,38.186329,POINT (116.12533 38.18633),130000,河北省
...,...,...,...,...,...,...
1562781,13178259,108.697861,30.928616,POINT (108.69786 30.92862),500000,重庆市
1562782,36912096,120.248520,30.182522,POINT (120.24852 30.18252),330000,浙江省
1562783,52187109,121.491493,30.921864,POINT (121.49149 30.92186),310000,上海市
1562784,5302301,119.688370,29.791410,POINT (119.68837 29.79141),330000,浙江省
